In [ ]:
import cv2
import glob
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator

import math
from keras import backend as K

In [ ]:
np.random.seed(7)
#from tensorflow import set_random_seed
#set_random_seed(2)

# Image Preprocessing

In [ ]:
#print('Train min=%.3f, max=%.3f' % (trainX.min(), trainX.max()))
#print('Test min=%.3f, max=%.3f' % (testX.min(), testX.max()))
# create generator (1.0/255.0 = 0.003921568627451)
plain_datagen = ImageDataGenerator()
scaled_datagen = ImageDataGenerator(rescale=1./255)
# prepare an iterators to scale images


Reference: https://machinelearningmastery.com/how-to-normalize-center-and-standardize-images-with-the-imagedatagenerator-in-keras/

In [ ]:
hr_image_raw_dict = {}
lr_image_raw_dict = {}




def load_images(standardize= False, resize = False, resize_width = None, resize_height = None):
    hr_image_list = glob.glob('/kaggle/input/super-image-resolution/Data/HR/*')
    
    for i in tqdm(hr_image_list):
        filename_wo_ext = i.split('/')[-1].split('.')[0]
        
        if resize:
            hr_image_raw_dict[filename_wo_ext] = cv2.resize(cv2.imread(i),fx = resize_height,
                                                           fy = resize_width)
        else:
            
            hr_image_raw_dict[filename_wo_ext] = cv2.imread(i)
        
        if standardize:
            hr_image_raw_dict[filename_wo_ext] = hr_image_raw_dict[filename_wo_ext]/255.0 
            
            
        
        
    lr_image_list = glob.glob('/kaggle/input/super-image-resolution/Data/LR/*')
    
    for i in tqdm(lr_image_list):
        filename_wo_ext = i.split('/')[-1].split('.')[0]
        if resize:
            lr_image_raw_dict[filename_wo_ext] = cv2.resize(cv2.imread(i),fx = resize_height,
                                                           fy = resize_width)
        else:
            
            lr_image_raw_dict[filename_wo_ext] = cv2.imread(i)
        #lr_image_raw_dict[filename_wo_ext] = cv2.imread(i)    
        
        if standardize:
            lr_image_raw_dict[filename_wo_ext] = lr_image_raw_dict[filename_wo_ext]/255.0 
            
        
    return hr_image_raw_dict, lr_image_raw_dict    

In [ ]:
hr_image_raw_dict, lr_image_raw_dict = load_images(standardize = False, resize = False, resize_width = None, resize_height = None)

In [ ]:
for i in range(4):
    a = str(np.random.randint(0,99))
    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(hr_image_raw_dict[a])
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(lr_image_raw_dict[a])
    plt.axis('off')

In [ ]:
lr_image_raw_dict[a].shape

In [ ]:
hr_image_raw_dict[a].shape

In [ ]:
def PSNR(y_true, y_pred):
    max_pixel = 1.0
    return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303

def SSIM(y_true, y_pred):
    return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))


def model_train_plot(history):
    
    
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    

    plt.plot(history.history['SSIM'])
    plt.plot(history.history['val_SSIM'])
    plt.title('model SSIM')
    plt.ylabel('SSIM')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['PSNR'])
    plt.plot(history.history['val_PSNR'])
    plt.title('model PSNR')
    plt.ylabel('PSNR')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()




def build_model():
    model = keras.Sequential()
    model.add(keras.Input(shape = (96,96,3)))
    model.add(layers.Conv2D(filters = 32, kernel_size= 3, padding = 'same', strides = 1,activation='relu'))
    model.add(layers.Conv2D(filters = 64, kernel_size= 3, padding = 'same', activation='relu'))
    model.add(layers.Conv2D(filters = 128, kernel_size= 9, padding = 'same', activation='relu' ))
    model.add(layers.Conv2D(filters = 64, kernel_size= 9, padding = 'same', activation='relu'))
    model.add(layers.UpSampling2D(size = (2,2)))
    model.add(layers.Conv2D(filters = 64, kernel_size= 3, padding = 'same', activation='relu'))
    model.add(layers.Conv2D(filters = 128, kernel_size= 9, padding = 'same', activation='relu'))
    model.add(layers.Conv2D(filters = 32, kernel_size= 3, padding = 'same', activation='relu'))
    model.add(layers.Conv2D(filters = 32, kernel_size= 3, padding = 'same', activation='relu'))
    model.add(layers.UpSampling2D(size = (2,2)))
    model.add(layers.Dense(3, activation="relu"))
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    
    model.compile(optimizer = opt, loss = 'MSE', metrics = [PSNR, 'accuracy', SSIM])
    model.summary()
    return model
    

In [ ]:
x_train = lr_image_raw_dict.values()
y_train = hr_image_raw_dict.values()
x_train = np.array(list(x_train))
y_train = np.array(list(y_train))

train_iterator_plain = plain_datagen.flow(x_train, y_train, batch_size=2)

model = build_model()

# Unscaled Data Modelling

In [ ]:
#model.fit(train_iterator_plain, batch_size=2, epochs=2000, verbose= 0 )

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

history = model.fit(x_train, y_train, batch_size=2, epochs=2000, verbose= 0,
                   callbacks=[earlystop_callback], validation_split = 0.2)


In [ ]:
history.history.keys()

# Model Performance

In [ ]:
model_train_plot(history)

In [ ]:
for i in range(4):
    a = str(np.random.randint(0,99))
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(hr_image_raw_dict[a])
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(lr_image_raw_dict[a])
    plt.axis('off')
    
    plt.subplot(1,3,3)
    plt.title('Super Resolved Image ', color = 'red', fontsize = 20)
    plt.imshow(model.predict(np.array(list(lr_image_raw_dict[a])).reshape(1, 96,96,3)).reshape(384,384,3))
    plt.axis('off')

# Scaled Data Visualisation
Reference: https://machinelearningmastery.com/how-to-evaluate-pixel-scaling-methods-for-image-classification/

In [ ]:
hr_image_raw_dict_scale, lr_image_raw_dict_scale = load_images(standardize = True, resize = False, resize_width = None, resize_height = None)

In [ ]:
for i in range(4):
    a = str(np.random.randint(0,99))
    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(hr_image_raw_dict_scale[a])
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(lr_image_raw_dict_scale[a])
    plt.axis('off')

In [ ]:
train_iterator_scaled = scaled_datagen.flow(x_train, y_train, batch_size=2)

# train_iterator_scaled = scaled_datagen.flow(x_train[0].reshape(1,96, 96, 3) , 
#                                             y_train[0].reshape(1, 384, 384, 3), 
#                                                                batch_size=1)
model = build_model()

# Scaled Data Modelling

In [ ]:
#model.fit(train_iterator_scaled, batch_size=2, epochs=2000, verbose= 2 )

x_train = lr_image_raw_dict_scale.values()
y_train = hr_image_raw_dict_scale.values()
x_train = np.array(list(x_train))
y_train = np.array(list(y_train))

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

history = model.fit(x_train, y_train, batch_size=2, epochs=2000, verbose= 0,
                   callbacks=[earlystop_callback], validation_split = 0.2)


# Model Performance

In [ ]:
model_train_plot(history)

In [ ]:
# # Store the data in X_train, y_train variables by iterating over the batches
# train_iterator_scaled.reset()
# X_train, Y_train = next(train_iterator_scaled)
# for i in tqdm(range(int(len(train_iterator_scaled))-1)): #1st batch is already fetched before the for loop.
#     img, label = next(train_iterator_scaled)
#     X_train = np.append(X_train, img, axis=0 )
#     Y_train = np.append(Y_train, label, axis=0)
# print(X_train.shape, Y_train.shape)

In [ ]:


# for i in range(4):
#     a = np.random.randint(0,99)
#     plt.figure(figsize=(20,10))
#     plt.subplot(1,3,1)
#     plt.title('High Resolution Imge', color = 'green', fontsize = 20)
#     plt.imshow(Y_train[a])
#     plt.axis('off')
#     plt.subplot(1,3,2)
#     plt.title('low Resolution Image ', color = 'black', fontsize = 20)
#     plt.imshow(X_train[a])
#     plt.axis('off')
    
#     plt.subplot(1,3,3)
#     plt.title('Super Resolved Image ', color = 'red', fontsize = 20)
#     plt.imshow(model.predict(np.array(list(X_train[a])).reshape(1, 96,96,3)).reshape(384,384,3))
#     plt.axis('off')

In [ ]:
for i in range(4):
    a = str(np.random.randint(0,99))
    plt.figure(figsize=(20,10))
    plt.subplot(1,3,1)
    plt.title('High Resolution Imge', color = 'green', fontsize = 20)
    plt.imshow(hr_image_raw_dict_scale[a])
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.title('low Resolution Image ', color = 'black', fontsize = 20)
    plt.imshow(lr_image_raw_dict_scale[a])
    plt.axis('off')
    
    plt.subplot(1,3,3)
    plt.title('Super Resolved Image ', color = 'red', fontsize = 20)
    plt.imshow(model.predict(np.array(list(lr_image_raw_dict_scale[a])).reshape(1, 96,96,3)).reshape(384,384,3))
    plt.axis('off')